In [1]:
import module1
import nltk
import numpy as np
import pandas as pd
import time

from sklearn import preprocessing

from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [5]:
filename = 'bank_train'
df = module1.read_df_from_csv(filename)
df_test = module1.read_df_from_csv('bank_test_etalon')

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
#we can add stop words

import pymorphy2
def tokenize(text):
    text = text.lower()
    text_list = nltk.word_tokenize(text)
    morph = pymorphy2.MorphAnalyzer()
    text_list = [word for word in text_list if word[0] != '@' and len(word) > 1 and word.isalpha()]
    text_list = [morph.parse(word)[0].normal_form for word in text_list]
    return text_list

In [7]:
#tf-idf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
j = 0
token_list = [i for i in df['text']]
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words=stop_words)
tfs = tfidf.fit_transform(token_list)

In [8]:
def df_filter(_df, _tfs, corporation):
    x_arr = []
    y_arr = []
    for i in range(len(_df)):
        label = _df[corporation][i]
        if label == '0' or label == '-1' or label == '1' or label == 0.0 or label == -1.0 or label == 1.0:
            x_arr.append(_tfs[i].toarray()[0])
            y_arr.append(int(label))
    X = np.array(x_arr)
    y = np.array(y_arr)
    print (len(X))
    print (len(y))
    return X, y

In [9]:
df

,Unnamed: 0,alfabank,date,gazprom,raiffeisen,rshb,sberbank,text,twitid,uralsib,vtb
0,1,0,1406224554,NaN,NaN,NaN,NaN,http://t.co/YEVHuvVGA1 Взять кредит тюмень аль...,492367586156630000,NaN,NaN
1,2,NaN,1406224691,NaN,NaN,NaN,NaN,Мнение о кредитной карте втб 24 http://t.co/SB...,492368160923070000,NaN,0
2,3,NaN,1406224798,NaN,0,NaN,NaN,«Райффайзенбанк»: Снижение ключевой ставки ЦБ ...,492368608346260000,NaN,NaN
3,4,NaN,1406225412,NaN,NaN,NaN,0,Современное состояние кредитного поведения в р...,492371181946030000,NaN,NaN
4,5,NaN,1406226283,NaN,NaN,NaN,1,@sawik_shuster @YevhenS Главное чтоб банки СБЕ...,492374836564750000,NaN,1
5,6,NaN,1406226450,NaN,NaN,NaN,NaN,http://t.co/Qr6JbSVTxY Оформить краткосрочный ...,492375537080600000,NaN,NaN
6,7,NaN,1406226505,NaN,NaN,NaN,NaN,Самый выгодный автокредит в втб 24 http://t.co...,492375766907510000,NaN,1
7,8,NaN,1406226607,NaN,NaN,NaN,+-,Похоже завтра Сбербанк и ВТБ присоединяется к ...,492376193791190000,NaN,+-
8,9,NaN,1406226960,NaN,NaN,NaN,0,Кредит иногородним в москве сбербанк http://t....,492377674535680000,NaN,NaN
9,10,NaN,1406226982,NaN,NaN,0,NaN,Кредитный калькулятор россельхозбанк чита http...,492377768232250000,NaN,NaN


In [10]:
companies = ['raiffeisen', 'alfabank', 'gazprom', 'raiffeisen', 'rshb', 'sberbank', 'uralsib', 'vtb']
X_train, y_train = df_filter(_df=df, _tfs=tfs, corporation=companies[0])
for companie in companies[1:]:
    X_curr, y_curr = df_filter(_df=df, _tfs=tfs, corporation=companie)
    X_train = np.append(X_train, X_curr, axis=0)
    y_train = np.append(y_train, y_curr)

278
278
562
562
372
372
278
278
929
929
2038
2038
82
82
1257
1257


In [11]:
companies = ['raiffeisen', 'alfabank', 'gazprom', 'raiffeisen', 'rshb', 'sberbank', 'uralsib', 'vtb']
X_test, y_test = df_filter(_df=df_test, _tfs=tfs, corporation=companies[0])
for companie in companies[1:]:
    X_curr, y_curr = df_filter(_df=df_test, _tfs=tfs, corporation=companie)
    if len(X_curr) > 0:
        X_test = np.append(X_test, X_curr, axis=0)
        y_test = np.append(y_test, y_curr)

150
150
536
536
137
137
150
150
99
99
2833
2833
60
60
651
651


In [12]:
print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print ('SVM: ')
start_time = time.time()
model = SVC()
model.fit(X_train, y_train)
t = int(time.time() - start_time)
print ('fit time = {0}m{1}sec'.format(t // 60, t % 60))
start_time = time.time()
y_predict = model.predict(X_test)
t = int(time.time() - start_time)
print ('predict time = {0}m{1}sec'.format(t // 60, t % 60))
print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
SVM: 
fit time = 4m39sec
predict time = 4m5sec


/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


f-macro: 0.2913363418303045
f-micro: 0.776213171577123


In [13]:
import my_score
func= lambda x,y: my_score.f_macro(my_score.cound_diff(x,y))
print ('strange f score - {0}'.format(func(y_predict, y_test)))

strange f score - 0.4568404947086574


In [14]:
print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print ('Decision Tree: ')
start_time = time.time()
model = DecisionTreeClassifier()
model.fit(X_train, y_train)
t = int(time.time() - start_time)
print ('fit time = {0} {1}'.format(t / 60, t % 60))
start_time = time.time()
y_predict = model.predict(X_test)
t = int(time.time() - start_time)
print ('predict time = {0} {1}'.format(t / 60, t % 60))
print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
print ('strange f score - {0}'.format(func(y_predict, y_test)))
print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
print ('AdaBoost DecisionTree max_depth = 2, n = 100: ')
start_time = time.time()
model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5), n_estimators=100,
learning_rate=1)
model.fit(X_train, y_train)
t = int(time.time() - start_time)
print ('fit time = {0} {1}'.format(t / 60, t % 60))
start_time = time.time()
y_predict = model.predict(X_test)
t = int(time.time() - start_time)
print ('predict time = {0} {1}'.format(t / 60, t % 60))
print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
print ('strange f score - {0}'.format(func(y_predict, y_test)))

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Decision Tree: 
fit time = 0.1 6
predict time = 0.0 0
f-macro: 0.33241627485286734
f-micro: 0.5853552859618717
strange f score - 0.4235376068860566
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
AdaBoost DecisionTree max_depth = 2, n = 100: 
fit time = 3.2666666666666666 16
predict time = 0.016666666666666666 1
f-macro: 0.3298327295503713
f-micro: 0.5899046793760832
strange f score - 0.4228240132659454


In [ ]:
#countVectorizer
start_time = time.time()
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(min_df=1, tokenizer=tokenize, stop_words=stop_words)
tfs4vec = vectorizer.fit_transform(token_list)
t = int(time.time() - start_time)
print ('TIME: {0}m {1}sec'.format(t / 60, t % 60))

In [ ]:
companies = ['raiffeisen']
for companie in companies:
    print('!!!!!!!!!!!!!!!!!')
    print(companie)
    print ('~~~~~~~~~~~~~~~~~')
    
    X, y = df_filter(_df = df, _tfs = tfs, corporation = companie)
    X4, y4 = df_filter(_df = df, _tfs = tfs4vec, corporation = companie)
    X_test, y_test = df_filter(_df = df_test, _tfs = tfs, corporation = companie)
    X4_test, y4_test = df_filter(_df = df_test, _tfs = tfs4vec, corporation = companie)
    
    index = int((X.shape[0] + X_test.shape[0]) * 0.2)
    if (index < X_test.shape[0]):
        X = np.vstack((X, X_test[index:]))
        X_test = X_test[:index]
        y = np.concatenate((y,y_test[index:]))
        y_test = y_test[:index]
        X4 = np.vstack((X4, X4_test[index:]))
        X4_test = X4_test[:index]
        y4 = np.concatenate((y4,y4_test[index:]))
        y4_test = y4_test[:index]
    
    print ('DATA SIZE: TRAIN = {0}, TEST= {1}'.format(X.shape[0], X_test.shape[0]))
    print ('Naive Bayes: ')
    start_time = time.time()
    model = MultinomialNB()
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    t = int(time.time() - start_time)
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model = MultinomialNB()
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('SVM: ')
    start_time = time.time()
    model = SVC()
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model = SVC()
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))  
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('SVM linear kernel: ')
    start_time = time.time()
    model = SVC(kernel = 'linear')
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model = SVC(kernel = 'linear')
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('kNN 10: ')
    start_time = time.time()
    model = KNeighborsClassifier(n_neighbors=10)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  KNeighborsClassifier(n_neighbors=10)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('kNN 5: ')
    start_time = time.time()
    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  KNeighborsClassifier(n_neighbors=5)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('kNN 3: ')
    start_time = time.time()
    model = KNeighborsClassifier(n_neighbors=3)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  KNeighborsClassifier(n_neighbors=3)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('kNN 7: ')
    start_time = time.time()
    model = KNeighborsClassifier(n_neighbors=7)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  KNeighborsClassifier(n_neighbors=7)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('kNN 30: ')
    start_time = time.time()
    model = KNeighborsClassifier(n_neighbors=30)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  KNeighborsClassifier(n_neighbors=30)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('Decision Tree: ')
    start_time = time.time()
    model = DecisionTreeClassifier()
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  DecisionTreeClassifier()
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('Decision Tree max depth = 5: ')
    start_time = time.time()
    model = DecisionTreeClassifier(max_depth = 5)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  DecisionTreeClassifier(max_depth = 5)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))
    print ('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
    print ('AdaBoost DecisionTree max_depth = 2, n = 100: ')
    start_time = time.time()
    model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=100,
    learning_rate=1)
    model.fit(X, y)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X_test)
    t = int(time.time() - start_time)
    print ('predict time = {0} {1}'.format(t / 60, t % 60))
    print ('f-macro: {0}'.format(f1_score(y_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y_test, y_predict, average = 'micro')))
    print('~~~~~count vectorizer~~~~~~')
    start_time = time.time()
    model =  AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=100,
    learning_rate=1)
    model.fit(X4, y4)
    t = int(time.time() - start_time)
    print ('fit time = {0} {1}'.format(t / 60, t % 60))
    start_time = time.time()
    y_predict = model.predict(X4_test)
    t = int(time.time() - start_time) 
    print ('predict time = {0} {1}'.format(t / 60, t % 60)) 
    print ('f-macro: {0}'.format(f1_score(y4_test, y_predict, average = 'macro')))
    print ('f-micro: {0}'.format(f1_score(y4_test, y_predict, average = 'micro')))